In [1]:
from pipelines import *
import mne
import random
%matplotlib widget

## Load data

In [2]:
subjects = ['Pilot_1', 'Pilot_2', 'Pilot_4', 'Pilot_5', 'Pilot_6', 'Pilot_7', 'Pilot_8', 'Pilot_9', 'Pilot_10', 'Pilot_11', 'Pilot_12', 'Pilot_13', 'Pilot_14', 'Pilot_15', 'Pilot_17', 'Pilot_18', 'Pilot_19', 'Pilot_20', 'Pilot_21']
PATTERN = 'Overlay'
SINGLEOBJ = True
subj_path = ['../../Experiments/data/Two_Obj/' + PATTERN + '/' + sub + '/' for sub in subjects]
nb_subj = len(subjects)
bads = [['A30', 'B25'], ['B25'], ['B25'], [], ['A31', 'B31'], ['B25'], ['A30', 'B25'], ['A30', 'B25'], ['B25'], ['B25', 'B26'], ['A30', 'B25'], ['B31'], ['B25', 'A23'], ['A30', 'B25'], ['B25'], ['B25'], ['A30', 'B25'], ['A30', 'B25'], ['B25']]
fsStim = 30
feats_path_folder = '../Feat_Multi/features_all/'

In [ ]:
LOAD_ONLY = False
ALL_NEW = True
GAZEFEATS = True
eeg_multisubj_list, eog_multisubj_list, feat_all_att_list, feat_all_unatt_list, gaze_multisubj_list, fs, len_seg_list = utils.load_data(subj_path, fsStim, bads, feats_path_folder, PATTERN, singleobj=False, LOAD_ONLY=LOAD_ONLY, ALL_NEW=ALL_NEW, GAZEFEATS=GAZEFEATS)
eeg_multisubj_list_SO, eog_multisubj_list_SO, feat_all_list_SO, _, gaze_multisubj_list_SO, fs, len_seg_list_SO = utils.load_data(subj_path, fsStim, bads, feats_path_folder, PATTERN, singleobj=True, LOAD_ONLY=LOAD_ONLY, ALL_NEW=ALL_NEW, GAZEFEATS=GAZEFEATS)

In [4]:
gaze_velocity_list = [utils.calcu_gaze_velocity(gaze) for gaze in gaze_multisubj_list]
gaze_coords_list = [gaze[:,0:2,:] for gaze in gaze_multisubj_list]
saccade_multisubj_list = [np.expand_dims(gaze[:,2,:], axis=1) for gaze in gaze_multisubj_list]
blink_multisubj_list = [np.expand_dims(gaze[:,3,:], axis=1) for gaze in gaze_multisubj_list]
saccade_multisubj_list = utils.refine_saccades(saccade_multisubj_list, blink_multisubj_list)
eog_velocity_list = [utils.calcu_gaze_vel_from_EOG(eog) for eog in eog_multisubj_list]
gaze_velocity_list = [utils.interpolate_blinks(gaze_velocity, blink) for gaze_velocity, blink in zip(gaze_velocity_list, blink_multisubj_list)]
gaze_coords_list = [utils.interpolate_blinks(gaze_coords, blink) for gaze_coords, blink in zip(gaze_coords_list, blink_multisubj_list)]
eog_velocity_list = [utils.interpolate_blinks(eog_velocity, blink) for eog_velocity, blink in zip(eog_velocity_list, blink_multisubj_list)] # blinks are not removed as cleanly as in the gaze data
mod_list = [eeg_multisubj_list, eog_multisubj_list, gaze_coords_list, gaze_velocity_list, eog_velocity_list, saccade_multisubj_list, feat_all_att_list, feat_all_unatt_list]

In [5]:
if SINGLEOBJ: # if include single object data in the analysis
    gaze_velocity_list_SO = [utils.calcu_gaze_velocity(gaze) for gaze in gaze_multisubj_list_SO]
    gaze_coords_list_SO = [gaze[:,0:2,:] for gaze in gaze_multisubj_list_SO]
    saccade_multisubj_list_SO = [np.expand_dims(gaze[:,2,:], axis=1) for gaze in gaze_multisubj_list_SO]
    blink_multisubj_list_SO = [np.expand_dims(gaze[:,3,:], axis=1) for gaze in gaze_multisubj_list_SO]
    saccade_multisubj_list_SO = utils.refine_saccades(saccade_multisubj_list_SO, blink_multisubj_list_SO)
    eog_velocity_list_SO = [utils.calcu_gaze_vel_from_EOG(eog) for eog in eog_multisubj_list_SO]
    gaze_velocity_list_SO = [utils.interpolate_blinks(gaze_velocity, blink) for gaze_velocity, blink in zip(gaze_velocity_list_SO, blink_multisubj_list_SO)]
    gaze_coords_list_SO = [utils.interpolate_blinks(gaze_coords, blink) for gaze_coords, blink in zip(gaze_coords_list_SO, blink_multisubj_list_SO)]
    eog_velocity_list_SO = [utils.interpolate_blinks(eog_velocity, blink) for eog_velocity, blink in zip(eog_velocity_list_SO, blink_multisubj_list_SO)] # blinks are not removed as cleanly as in the gaze data
    mod_list = [eeg_multisubj_list, eog_multisubj_list, gaze_coords_list, gaze_velocity_list, eog_velocity_list, saccade_multisubj_list, feat_all_att_list, feat_all_unatt_list, eeg_multisubj_list_SO, eog_multisubj_list_SO, gaze_coords_list_SO, gaze_velocity_list_SO, eog_velocity_list_SO, saccade_multisubj_list_SO, feat_all_list_SO]

In [6]:
# Check the alignment between eog and gaze. The synchronization is good if the peaks of two signals (eye blinks) are aligned.
# subj_to_check = 'Pilot_1'
# subj_ID = subjects.index(subj_to_check)
# utils.check_alignment(subj_ID, eog_multisubj_list, gaze_multisubj_list, nb_points=1500)

In [7]:
RemoveSC = True # remove shot cuts
if RemoveSC:
    mod_list = [[utils.remove_shot_cuts(d, fs) for d in sublist] for sublist in mod_list]
if not SINGLEOBJ:
    [eeg_multisubj_list, eog_multisubj_list, gaze_coords_list, gaze_velocity_list, eog_velocity_list, saccade_multisubj_list, feat_all_att_list, feat_all_unatt_list] = mod_list
else:
    [eeg_multisubj_list, eog_multisubj_list, gaze_coords_list, gaze_velocity_list, eog_velocity_list, saccade_multisubj_list, feat_all_att_list, feat_all_unatt_list, eeg_multisubj_list_SO, eog_multisubj_list_SO, gaze_coords_list_SO, gaze_velocity_list_SO, eog_velocity_list_SO, saccade_multisubj_list_SO, feat_all_list_SO] = mod_list

In [8]:
# Features being used in the analysis: object optical flow
feat_att_list = feat_all_att_list
feat_unatt_list = feat_all_unatt_list
# Dictionary for modalities
modal_dict = {'EEG': eeg_multisubj_list, 'EOG': eog_multisubj_list, 'GAZE': gaze_coords_list, 'GAZE_V': gaze_velocity_list
              , 'EOG_V': eog_velocity_list, 'SACC': saccade_multisubj_list}

if SINGLEOBJ:
    feat_list_SO = feat_all_list_SO
    modal_dict_SO = {'EEG': eeg_multisubj_list_SO, 'EOG': eog_multisubj_list_SO, 'GAZE': gaze_coords_list_SO, 'GAZE_V': gaze_velocity_list_SO
                  , 'EOG_V': eog_velocity_list_SO, 'SACC': saccade_multisubj_list_SO}

In [9]:
# Parameters in CCA; L_EEG and offset_EEG are not only used for EEG but also for other modalities
L_EEG = 3 
L_Stim = int(fsStim/2) 
offset_EEG = 1 
offset_Stim = 0 
trial_len_list = list(range(15, 75, 15))

In [10]:
# Folders for saving figures and tables; Currently there are no figures for modalities other than EEG, but the folders are still there.
figure_dirs = {}
table_dirs = {}
CLEAR = False # clear the content if set to True
for data_type in modal_dict.keys():
    figure_path = f'figures_frw/{PATTERN}/{data_type}/'
    utils.create_dir(figure_path, CLEAR)
    figure_dirs[data_type] = figure_path
    table_path = f'tables_frw/{PATTERN}/{data_type}/'
    utils.create_dir(table_path, CLEAR)
    table_dirs[data_type] = table_path

In [11]:
new_subj = ['Pilot_1']
new_subj_idx = [subjects.index(sub) for sub in new_subj]
CALCU_NEW_SUBJ_ONLY = False
Subj_Set = new_subj_idx if CALCU_NEW_SUBJ_ONLY else range(nb_subj)

In [12]:
len_sec = [feat_att.shape[0]/fs for feat_att in feat_att_list]
print(np.max(len_sec), np.min(len_sec))
print(np.mean(len_sec), np.std(len_sec))

300.0 78.0
165.86666666666665 75.86241723809194


## Visual attention decoding: Discriminating between attended and unattended segments

In [ ]:
# Get the results for either all subjects or new subjects
for Subj_ID in Subj_Set:
    for modal in modal_dict.keys():
        PLOT = True if modal == 'EEG' else False
        pipe_att_or_unatt_LVO(Subj_ID, modal_dict[modal], feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT=True, PLOT=PLOT, figure_dir=figure_dirs[modal], SAVERES=True, table_dir=table_dirs[modal], OVERWRITE=True)
        # pipe_att_or_unatt_LVO(Subj_ID, modal_dict[modal], feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT=False, PLOT=PLOT, figure_dir=figure_dirs[modal], SAVERES=True, table_dir=table_dirs[modal], OVERWRITE=True)
        pipe_compete_trials_LVO(Subj_ID, modal_dict[modal], feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dirs[modal], OVERWRITE=True)

### If remove saccades

In [ ]:
nested_data = [modal_dict['EEG'], feat_att_list, feat_unatt_list, modal_dict['SACC']]
nested_aug_data = [modal_dict_SO['EEG'], feat_list_SO, None, modal_dict_SO['SACC']]
pipe_saccade(nested_data, nested_aug_data, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dir=table_dirs['EEG'], OVERWRITE=True, SYNMASK=False)

In [ ]:
for run in range(1, 11):
    name = f'Run{run}-ObjFlow'
    nested_data = [modal_dict['EEG'], feat_att_list, feat_unatt_list, modal_dict['SACC']]
    nested_aug_data = [modal_dict_SO['EEG'], feat_list_SO, None, modal_dict_SO['SACC']]
    pipe_saccade(nested_data, nested_aug_data, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dir=table_dirs['EEG'], OVERWRITE=True, SYNMASK=True, feat_name=name)

### If use a subset of EEG channels

In [ ]:
# Note: lz is added to the parietal_occipital region
region_dict = {
    'frontal': ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8'],
    'frontal_central': ['FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6'],
    'temporal': ['FT7', 'T7', 'TP7', 'FT8', 'T8', 'TP8'],
    'central_parietal': ['CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'Pz', 'CPz', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10'],
    'parietal_occipital': ['PO7', 'PO3', 'POz', 'PO4', 'PO8', 'O1', 'Oz', 'O2', 'Iz'],
    'selected': ['PO7', 'PO4', 'Oz', 'O1']
}

In [ ]:
montage = mne.channels.make_standard_montage('biosemi64')
for region in region_dict.keys():
    vis_collection = region_dict[region]
    vis_collection_indices = [montage.ch_names.index(ch) for ch in vis_collection]
    data = modal_dict['EEG']
    eeg_subset = [eeg[:,vis_collection_indices,:] for eeg in data]
    for Subj_ID in Subj_Set:
        # pipe_att_or_unatt_LVO(Subj_ID, eeg_subset, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT=True, SAVERES=True, table_dir=table_dirs['EEG'], OVERWRITE=True, feat_name=region+'-OF')
        pipe_compete_trials_LVO(Subj_ID, eeg_subset, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dirs['EEG'], OVERWRITE=True, feat_name=region+'-OF')
        pipe_mm_trials_LVO(Subj_ID, eeg_subset, feat_att_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dirs['EEG'], MATCHATT=True, OVERWRITE=True, feat_name=region+'-OF')

In [ ]:
montage = mne.channels.make_standard_montage('biosemi64')
for run in range(1, 11):
    # remove the channels in the parietal_occipital region
    collection_refined = [ch for ch in montage.ch_names if ch not in region_dict['parietal_occipital']]
    vis_collection = random.sample(collection_refined, 4)
    print(vis_collection, '####################################################')
    vis_collection_indices = [montage.ch_names.index(ch) for ch in vis_collection]
    data = modal_dict['EEG']
    eeg_subset = [eeg[:,vis_collection_indices,:] for eeg in data]
    for Subj_ID in Subj_Set:
        # pipe_att_or_unatt_LVO(Subj_ID, eeg_subset, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT=True, SAVERES=True, table_dir=table_dirs['EEG'], OVERWRITE=True, feat_name=f'Run-{run}-Rand4Ref-OF')
        pipe_compete_trials_LVO(Subj_ID, eeg_subset, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dirs['EEG'], OVERWRITE=True, feat_name=f'Run-{run}-Rand4Ref-OF')

## Match-Mismatch: Discriminating between match and mismatch segments

In [ ]:
# Get the results for either all subjects or new subjects
for Subj_ID in Subj_Set:
    for modal in modal_dict.keys():
        pipe_mm_trials_LVO(Subj_ID, modal_dict[modal], feat_att_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dirs[modal], MATCHATT=True, OVERWRITE=True)
        # pipe_mm_trials_LVO(Subj_ID, modal_dict[modal], feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dirs[modal], MATCHATT=False, OVERWRITE=True)

## Making use of multi-modal data

### Stacking different modalities

In [14]:
comb_dict = {'EEG+EOG': utils.stack_modal([modal_dict['EEG'], modal_dict['EOG']]), 
             'EEG+GAZE_V': utils.stack_modal([modal_dict['EEG'], modal_dict['GAZE_V']]), 
             'EOG+GAZE_V': utils.stack_modal([modal_dict['EOG'], modal_dict['GAZE_V']]), 
             'EEG+EOG+GAZE_V': utils.stack_modal([modal_dict['EEG'], modal_dict['EOG'], modal_dict['GAZE_V']]),
             'EOG+GAZE+GAZE_V': utils.stack_modal([modal_dict['EOG'], modal_dict['GAZE'], modal_dict['GAZE_V']])}

In [ ]:
for comb, modal in comb_dict.items():
    table_path = f'tables_frw/{PATTERN}/{comb}/'
    utils.create_dir(table_path, CLEAR)
    for Subj_ID in Subj_Set:
        pipe_att_or_unatt_LVO(Subj_ID, modal[0], feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, dim_list_EEG=modal[1], TRAIN_WITH_ATT=True, SAVERES=True, table_dir=table_path, OVERWRITE=True)
        pipe_compete_trials_LVO(Subj_ID, modal[0], feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_path, dim_list_EEG=modal[1], OVERWRITE=True, nb_comp_into_account=2)
        pipe_mm_trials_LVO(Subj_ID, modal[0], feat_att_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_path, MATCHATT=True, dim_list_EEG=modal[1], OVERWRITE=True, nb_comp_into_account=2)

### GCCA with multi-modal data [Not as good as the previous one]
Potential problem: the correlation between, e.g., EEG and GAZE_V, dominates the ISC when the feature is less informative. 
Therefore the ISCs under attended and unattended conditions are not well separated.

In [ ]:
Subj_ID = 0
nested_datalist = [modal_dict['EEG'], modal_dict['GAZE_V'], feat_att_list, feat_unatt_list]
L_list = [L_EEG, L_EEG, L_Stim, L_Stim]
offset_list = [offset_EEG, offset_EEG, offset_Stim, offset_Stim]
# nested_datalist = [modal_dict['EEG'], feat_att_list, feat_unatt_list]
# L_list = [L_EEG, L_Stim, L_Stim]
# offset_list = [offset_EEG, offset_Stim, offset_Stim]
pipe_att_or_unatt_gcca(Subj_ID, nested_datalist, fs, L_list, offset_list, nb_comp_into_account=3)

### Regressing out other modalities from EEG (Partial CCA)

In [15]:
reg_dict = {'EEG-EOG': utils.regress_out_confounds(modal_dict['EEG'], feat_att_list, feat_unatt_list, modal_dict['EOG'], L_EEG, L_Stim, offset_EEG, offset_Stim),
             'EEG-GAZE_V': utils.regress_out_confounds(modal_dict['EEG'], feat_att_list, feat_unatt_list, modal_dict['GAZE_V'], L_EEG, L_Stim, offset_EEG, offset_Stim),
             'EEG-EOG&GAZE_V': utils.regress_out_confounds(modal_dict['EEG'], feat_att_list, feat_unatt_list, comb_dict['EOG+GAZE_V'][0], L_EEG, L_Stim, offset_EEG, offset_Stim),
             'EEG-EOG&GAZE&GAZE_V': utils.regress_out_confounds(modal_dict['EEG'], feat_att_list, feat_unatt_list, comb_dict['EOG+GAZE+GAZE_V'][0], L_EEG, L_Stim, offset_EEG, offset_Stim)}

In [ ]:
for reg, modal in reg_dict.items():
    table_path = f'tables_frw/{PATTERN}/{reg}/'
    utils.create_dir(table_path, CLEAR=False)
    PLOT = (reg.split('-')[0] == 'EEG')
    if PLOT:
        figure_path = f'figures_frw/{PATTERN}/{reg}/'
        utils.create_dir(figure_path, CLEAR=False)
    else:
        figure_path = None
    for Subj_ID in Subj_Set:
        pipe_att_or_unatt_LVO(Subj_ID, modal[0], modal[1], modal[2], fs, L_EEG=1, L_Stim=1, offset_EEG=0, offset_Stim=0, TRAIN_WITH_ATT=True, eeg_ori_list=modal_dict['EEG'], PLOT=PLOT, figure_dir=figure_path, SAVERES=True, table_dir=table_path, OVERWRITE=True)
        pipe_compete_trials_LVO(Subj_ID, modal[0], modal[1], modal[2], fs, L_EEG=1, L_Stim=1, offset_EEG=0, offset_Stim=0, trial_len_list=trial_len_list, table_dir=table_path, OVERWRITE=True)

In [ ]:
for reg, modal in reg_dict.items():
    table_path = f'tables_frw/{PATTERN}/{reg}/'
    utils.create_dir(table_path, CLEAR=False)
    PLOT = (reg.split('-')[0] == 'EEG')
    if PLOT:
        figure_path = f'figures_frw/{PATTERN}/{reg}/'
        utils.create_dir(figure_path, CLEAR=False)
    else:
        figure_path = None
    for Subj_ID in Subj_Set:
        pipe_att_or_unatt_LVO(Subj_ID, modal[0], modal[1], modal[2], fs, L_EEG=1, L_Stim=1, offset_EEG=0, offset_Stim=0, TRAIN_WITH_ATT=True, eeg_ori_list=modal_dict['EEG'], PLOT=PLOT, figure_dir=figure_path, SAVERES=True, table_dir=table_path, OVERWRITE=True)
        pipe_compete_trials_LVO(Subj_ID, modal[0], modal[1], modal[2], fs, L_EEG=1, L_Stim=1, offset_EEG=0, offset_Stim=0, trial_len_list=trial_len_list, table_dir=table_path, OVERWRITE=True)

In [ ]:
# Get the results for either all subjects or new subjects
for Subj_ID in Subj_Set:
    for reg, modal in reg_dict.items():
        table_path = f'tables_frw/{PATTERN}/{reg}/'
        pipe_mm_trials_LVO(Subj_ID, modal[0], modal[1], fs, 1, 1, 0, 0, trial_len_list, table_path, MATCHATT=True, OVERWRITE=True)
        # pipe_mm_trials_LVO(Subj_ID, modal[0], modal[2], fs, 1, 1, 0, 0, trial_len_list, table_path, MATCHATT=False, OVERWRITE=True)

## GCCA

In [ ]:
GCCA_table_path = f'tables_frw/{PATTERN}/GCCA/'
GCCA_figure_path = f'figures_frw/{PATTERN}/GCCA/'
utils.create_dir(GCCA_table_path, CLEAR=False)
utils.create_dir(GCCA_figure_path, CLEAR=False)

### Single-object dataset

In [ ]:
nested_datalist = list(modal_dict_SO.values()) 
mod_name_list = list(modal_dict_SO.keys()) 
L_list = [5]*len(nested_datalist)
offset_list = [2]*len(nested_datalist)
W_list = [None]*len(nested_datalist)
nested_dimlist = [None]*len(nested_datalist)
pipe_GCCA(nested_datalist, fs, L_list, offset_list, mod_name_list, W_list, nested_dimlist, GCCA_figure_path, GCCA_table_path, SINGLEOBJ=True, OVERWRITE=True)

In [ ]:
comb_dict_SO = {'EEG+EOG': utils.stack_modal([modal_dict_SO['EEG'], modal_dict_SO['EOG']]), 
             'EEG+GAZE_V': utils.stack_modal([modal_dict_SO['EEG'], modal_dict_SO['GAZE_V']]), 
             'EOG+GAZE_V': utils.stack_modal([modal_dict_SO['EOG'], modal_dict_SO['GAZE_V']]), 
             'EEG+EOG+GAZE_V': utils.stack_modal([modal_dict_SO['EEG'], modal_dict_SO['EOG'], modal_dict_SO['GAZE_V']])}

In [ ]:
reg_gcca_dict_SO = {'EEG-EOG': utils.regress_out_confounds(modal_dict_SO['EEG'], feat_list_SO, feat_list_SO, modal_dict_SO['EOG'], L_data=5, L_Stim=1, offset_data=2, offset_Stim=0),
             'EEG-GAZE_V': utils.regress_out_confounds(modal_dict_SO['EEG'], feat_list_SO, feat_list_SO, modal_dict_SO['GAZE_V'], L_data=5, L_Stim=1, offset_data=2, offset_Stim=0),
             'EEG-EOG&GAZE_V': utils.regress_out_confounds(modal_dict_SO['EEG'], feat_list_SO, feat_list_SO, comb_dict_SO['EOG+GAZE_V'][0], L_data=5, L_Stim=1, offset_data=2, offset_Stim=0)}

In [ ]:
nested_datalist = [modal[0] for modal in reg_gcca_dict_SO.values()] 
mod_name_list = list(reg_gcca_dict_SO.keys()) 
L_list = [1]*len(nested_datalist)
offset_list = [0]*len(nested_datalist)
W_list = [None]*len(nested_datalist)
nested_dimlist = [None]*len(nested_datalist)
pipe_GCCA(nested_datalist, fs, L_list, offset_list, mod_name_list, W_list, nested_dimlist, GCCA_figure_path, GCCA_table_path, SINGLEOBJ=True, OVERWRITE=True, FM_ORG=[5 ,2])

### Overlaid-object dataset

In [ ]:
nested_datalist = list(modal_dict.values())
mod_name_list = list(modal_dict.keys()) 
L_list = [5]*len(nested_datalist)
offset_list = [2]*len(nested_datalist)
W_list = [None]*len(nested_datalist)
nested_dimlist = [None]*len(nested_datalist)
pipe_GCCA(nested_datalist, fs, L_list, offset_list, mod_name_list, W_list, nested_dimlist, GCCA_figure_path, GCCA_table_path, SINGLEOBJ=False, OVERWRITE=True)

In [ ]:
reg_gcca_dict = {'EEG-EOG': utils.regress_out_confounds(modal_dict['EEG'], feat_att_list, feat_unatt_list, modal_dict['EOG'], L_data=5, L_Stim=1, offset_data=2, offset_Stim=0),
             'EEG-GAZE_V': utils.regress_out_confounds(modal_dict['EEG'], feat_att_list, feat_unatt_list, modal_dict['GAZE_V'], L_data=5, L_Stim=1, offset_data=2, offset_Stim=0),
             'EEG-EOG&GAZE_V': utils.regress_out_confounds(modal_dict['EEG'], feat_att_list, feat_unatt_list, comb_dict['EOG+GAZE_V'][0], L_data=5, L_Stim=1, offset_data=2, offset_Stim=0)}

In [ ]:
nested_datalist = [modal[0] for modal in reg_gcca_dict.values()] 
mod_name_list = list(reg_gcca_dict.keys()) 
L_list = [1]*len(nested_datalist)
offset_list = [0]*len(nested_datalist)
W_list = [None]*len(nested_datalist)
nested_dimlist = [None]*len(nested_datalist)
pipe_GCCA(nested_datalist, fs, L_list, offset_list, mod_name_list, W_list, nested_dimlist, GCCA_figure_path, GCCA_table_path, SINGLEOBJ=False, OVERWRITE=True, FM_ORG=[5 ,2])

### GCCA as preprocessing

In [ ]:
para_gcca = [5, 2]
para_cca_eeg = [1, 0]
para_cca_stim = [L_Stim, offset_Stim]
pipe_GCCA_preprocessed(modal_dict['EEG'], feat_att_list, feat_unatt_list, fs, para_gcca, para_cca_eeg, para_cca_stim, table_dir=table_dirs['EEG'], figure_dir=figure_dirs['EEG'], OVERWRITE=True)

In [ ]:
para_gcca = [5, 2]
para_cca_eeg = [1, 0]
para_cca_stim = [L_Stim, offset_Stim]
pipe_GCCA_preprocessed(modal_dict['GAZE_V'], feat_att_list, feat_unatt_list, fs, para_gcca, para_cca_eeg, para_cca_stim, n_components_GCCA=5, n_components_CCA=2, nb_comp_kept_grid=range(2,4), table_dir=table_dirs['GAZE_V'], figure_dir=figure_dirs['GAZE_V'], OVERWRITE=True)